In [35]:
# A. 	Basic Machine learning models
#{(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)} are divided into 6 classes labelled into [0, 5]. 

# import raw train test audio files 
Folder = r'C:\Users\zz587-admin\Documents\lecutures\ECE 4200 2021\kaggle\spoken-digit-pair-recognition'

import numpy as np
from scipy.io import wavfile
import pandas
train_data=[]
test_data=[]
for i in range (90000):
    rate,data = wavfile.read(Folder + r'\train\train_new\train_' + str(i)+'.wav')
    data = np.array(data)
    train_data.append(data)

for i in range (24750):
    rate,data_temp = wavfile.read(Folder + r'\test\test_new\test_' + str(i)+'.wav')
    data_temp = np.array(data_temp)
    test_data.append(data_temp)




In [5]:
# generate class  from csv file for all training cases 

Trainfile = pandas.read_csv(Folder + r'\train.csv')
Trainfile.loc[Trainfile['Label'] == 21, 'class'] = 0
Trainfile.loc[Trainfile['Label'] == 31, 'class'] = 1
Trainfile.loc[Trainfile['Label'] == 41, 'class'] = 2
Trainfile.loc[Trainfile['Label'] == 32, 'class'] = 3
Trainfile.loc[Trainfile['Label'] == 42, 'class'] = 4
Trainfile.loc[Trainfile['Label'] == 43, 'class'] = 5

y_train_all = Trainfile['class'].astype('int')
y_train_all=np.array(y_train_all) 
train_data=np.array(train_data)
test_data=np.array(test_data) 

In [15]:
# data preprocessing : tranform by FFT 
from sklearn import preprocessing
from scipy import signal
train_data_fft=[]
test_data_fft=[]
for i in range (90000):
    _, Pxx_den = signal.periodogram(train_data[i,:], rate)
    train_data_fft.append(Pxx_den)

for i in range (24750):
    _, Pxx_den = signal.periodogram(test_data[i,:], rate)
    test_data_fft.append(Pxx_den)

In [27]:
# data preprocessing : Standard  Scaler 
train_data_scaler=preprocessing.StandardScaler().fit(train_data)
test_data_scaler=preprocessing.StandardScaler().fit(test_data)

In [23]:
# Save FFT transformed data 
train_data_fft=np.array(train_data_fft) 
test_data_fft=np.array(test_data_fft) 
np.save('fft_train_data.npy',train_data_fft)
np.save('fft_test_data.npy',test_data_fft)
np.save('y_train_data.npy',y_train_all)

In [2]:
# import FFT transformed data 
import numpy as np
train_data_fft=np.load('fft_train_data.npy')
test_data_fft=np.load('fft_test_data.npy')

In [15]:
# split all training cases into training and testing part. 
# get testing accuracy to evaluate the performance by different ML models 
# LogisticRegression,DecisionTree,svm,GaussianNB,RandomForest,xgboost are tried 

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(train_data_fft, y_train_all, test_size=0.33)

clf1 = LogisticRegression(random_state=0,C=0.7,max_iter=10000)
clf2 = DecisionTreeClassifier(random_state=0)

clf3 = GaussianNB()
clf4= svm.SVC(kernel="linear", C=0.7)
clf5 = RandomForestClassifier(random_state=0)
clf6 = BaggingClassifier(base_estimator=clf1,n_estimators=10, random_state=0)

clf7 = xgb.XGBClassifier(n_estimators=5)
 
# clf=clf7.fit(X_train, y_train)
# y_pred_test = clf.predict(X_test)
model=clf1.fit(X_train, y_train)
y_pred_test = model.predict(X_test)

y_pred_test=np.array(y_pred_test)
print("Number of mislabeled points out of a total %d points : %d"      % (X_test.shape[0], (y_test != y_pred_test).sum()))
print("accuracy %d "      %  float((y_test == y_pred_test).sum()/X_test.shape[0]))


Number of mislabeled points out of a total 29700 points : 0
accuracy 1 


In [34]:
# k-fold validation over all training data 
# get average accuracy over all training data 
# test on LogisticRegression,DecisionTree,svm,GaussianNB 
# Output is presented in report 

from sklearn.model_selection import KFold
def kFoldCV (x,y,model):
    x=train_data_fft
    y=y_train_all
    
    rkf = KFold(n_splits=3)
    acc_all=[]

    for train_ind, test_ind in rkf.split(x,y):
        X_train=x[train_ind] 
        X_test=x[test_ind]
        y_train=y[train_ind] 
        y_test=y[test_ind]
        model_fitted=model.fit(X_train, y_train)
        y_pred_test =model_fitted.predict(X_test)
        acc=float((y_test == y_pred_test).sum()/X_test.shape[0])
        acc_all.append(acc)
    acc_all=np.array(acc_all)
    acc_mean=np.mean(acc_all)
    return acc_mean

x=train_data_fft
y=y_train_all

model_LogReg = LogisticRegression(random_state=0,C=0.7,max_iter=10000)
model_tree = DecisionTreeClassifier(random_state=0)

model_NB = GaussianNB()
model_svm= svm.SVC(kernel="linear", C=0.7)
acc_LogReg=kFoldCV(x,y,model_LogReg)
acc_NB=kFoldCV(x,y,model_NB)
acc_tree=kFoldCV(x,y,model_tree)
acc_svm=kFoldCV(x,y,model_svm)



In [37]:
# Finally output results using trained mode on new testing data 


def outputCSV(model,x_train,y_train,x_test,Name):
    
    y_test_new=model.fit(x_train, y_train_all).predict(x_test)
    
    y_test_new=np.array(y_test_new)
    
    label=np.zeros((24750,1))
    #label=np.zeros((27250,1))


    label[y_test_new==0]=21
    label[y_test_new==1]=31
    label[y_test_new==2]=41
    label[y_test_new==3]=32
    label[y_test_new==4]=42
    label[y_test_new==5]=43


    import csv

    with open(Name+'.csv', 'w', newline='') as file:
        fieldnames = ['ID', 'Label']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(24750):
            writer.writerow({'ID': i, 'Label': int(label[i])})
      

In [39]:
# outputCSV(model_LogReg,x,y,test_data_fft,'LogReg1')
# outputCSV(model_svm,x,y,test_data_fft,'svm1')
outputCSV(model_tree,x,y,test_data_fft,'tree1')
outputCSV(model_NB,x,y,test_data_fft,'NB1')